# Import Dependencies

In [20]:
import os
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from collections import OrderedDict
from tqdm import tqdm
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch import optim
import time
from torch.autograd import Variable
%matplotlib inline

In [21]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# Load Data

In [22]:
data_dir = "mliii-assignment2" # Path to data directory
labels = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
assert(len(os.listdir(os.path.join(data_dir, 'train'))) == len(labels))

In [23]:
le = LabelEncoder()
labels.breed = le.fit_transform(labels.breed)
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,19
1,001513dfcb2ffafc82cccf4d8bbaba97,37
2,001cdf01b096e06d78e9e5112d419397,85
3,00214f311d5d2247d5dfe4fe24b2303d,15
4,0021f9ceb3235effd7fcde7f7538ed62,49


In [24]:
X = labels.id
y = labels.breed

X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.4, random_state=SEED, stratify=y)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=SEED, stratify=y_valid)

In [25]:
X_train

44      01268f0007876a692907bda89468184c
5036    7f5f6d0df6e42003bc34b803a2eaa860
2022    323e6e009ca300b37537df232fa8b734
6667    a75dc89b84b4986afa0365ae9f734681
7476    bbb8f439fd4a3563bfd4adf9a13f613d
                      ...               
501     0c19867277e6c96ad8f487b4fe343ff9
595     0ea00d73664139c52ee1eaabc77ea478
3157    4e00cf013a2e530e75a99a4d8e631d1e
498     0c0955cddf9488d30e6e8d4ee7c18856
7103    b22e3e2987ce4e7b599d109df6ef9d98
Name: id, Length: 6133, dtype: object

In [26]:
from torchvision.transforms import Resize, ToTensor, Normalize

means = np.load('means.npy')
stds = np.load('stds.npy')
class Dataset_Interpreter(Dataset):
    def __init__(self, data_path, file_names, labels=None):
        self.data_path = data_path
        self.file_names = file_names
        self.labels = labels
        self.resize = Resize((224,224))
        self.normalize = Normalize(means, stds)
        self.trans = transforms.Compose([
            transforms.AugMix(),   
            transforms.ToTensor()
        ])
        
    def __len__(self):
        return (len(self.file_names))
    
    def __getitem__(self, idx):
        img_name = f'{self.file_names.iloc[idx]}.jpg'
        full_address = os.path.join(self.data_path, img_name)
        image = Image.open(full_address)
        label = self.labels.iloc[idx]
        image = self.resize(image)
        image = self.trans(image)
        image = self.normalize(image)
#         if self.transforms is not None:
#             image = self.transforms(image)
        
        return image, label

class Dataset_Interpreter_crop(Dataset):
    def __init__(self, data_path, file_names, labels=None):
        self.data_path = data_path
        self.file_names = file_names
        self.labels = labels
        self.resize = Resize((224, 224))
        self.normalize = Normalize(means, stds)
        self.trans = transforms.Compose([
            transforms.TenCrop((100, 100)),
            transforms.Lambda(lambda crops: [transforms.Resize((224, 224))(crop) for crop in crops]),
            transforms.ColorJitter(),
            transforms.Lambda(lambda crops: torch.stack([transforms.PILToTensor()(crop).float() for crop in crops])),
        ])
        self.to_tensor = ToTensor()
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        img_name = f'{self.file_names.iloc[idx]}.jpg'
        full_address = os.path.join(self.data_path, img_name)
        image = Image.open(full_address)
        label = self.labels.iloc[idx]
        image = self.resize(image)
        cropped_images = self.trans(image)
        transformed_images = self.normalize(cropped_images)
        
        return transformed_images, label
    
class Dataset_Interpreter_valid_test(Dataset):
    def __init__(self, data_path, file_names, labels=None):
        self.data_path = data_path
        self.file_names = file_names
        self.labels = labels
        self.resize = Resize((224,224))
        self.normalize = Normalize(means, stds)
        self.trans = transforms.Compose([   
            transforms.ToTensor()
        ])
        
    def __len__(self):
        return (len(self.file_names))
    
    def __getitem__(self, idx):
        img_name = f'{self.file_names.iloc[idx]}.jpg'
        full_address = os.path.join(self.data_path, img_name)
        image = Image.open(full_address)
        label = self.labels.iloc[idx]
        image = self.resize(image)
        image = self.trans(image)
        image = self.normalize(image)
#         if self.transforms is not None:
#             image = self.transforms(image)
        
        return image, label

In [27]:
def plot_images(images):

    n_images = len(images)

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize=(20,10))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        ax.set_title(f'{le.inverse_transform([images[i][1]])}')
        ax.imshow(np.array(images[i][0]))
        ax.axis('off')
N_IMAGES = 9

train_data = Dataset_Interpreter(data_path=data_dir+'/train/', file_names=X_train, labels=y_train)
crop_data = Dataset_Interpreter_crop(data_path=data_dir+'/train/', file_names=X_train, labels=y_train)
valid_data = Dataset_Interpreter_valid_test(data_path=data_dir+'/train/', file_names=X_valid, labels=y_valid)
test_data = Dataset_Interpreter_valid_test(data_path=data_dir+'/train/', file_names=X_test, labels=y_test)
images = [(image, label) for image, label in [train_data[i] for i in range(N_IMAGES)]] 
#plot_images(images)

In [28]:
train_data[0][0].size()

torch.Size([3, 224, 224])

In [29]:
crop_data[0][0].size()

torch.Size([10, 3, 224, 224])

In [30]:
len(train_data)

6133


def combine_crop(data):
    reformatted_data = []  # Initialize an empty list to store reformatted data
    
    for i in range(len(data)):
        for j in range(10):
            image = data[i][0][j].to('cuda')
            label = data[i][1]
            reformatted_data.append((image, label))  # Append the image and label as a tuple
    
    return reformatted_data  # Return the list of (image, label) tuples

crop = combine_crop(crop_data)
len(crop)

channel_1_mean = np.mean([train_data[x][0][:,:,0] for x in range(0, len(train_data))])

channel1 = [train_data[x][0][:,:,0] for x in range(0, len(train_data))]

channel2 = [train_data[x][0][:,:,1] for x in range(0, len(train_data))]
channel3 = [train_data[x][0][:,:,2] for x in range(0, len(train_data))]

channel_2_mean = np.mean(channel2)
channel_3_mean = np.mean(channel3)
channel_1_std = np.std(channel1)
channel_2_std = np.std(channel2)
channel_3_std = np.std(channel3)

means = np.array([channel_1_mean, channel_2_mean, channel_3_mean])
stds = np.array([channel_1_std, channel_2_std, channel_3_std])
np.save('stds', stds)
np.save('means', means)

# Data Preprocessing

In [31]:
# # Data augmentation and normalization for training
# # Just normalization for validation
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(means, stds)
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(means, stds)
#     ]),
# }

# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])
#                   for x in ['train', 'val']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
#                                              shuffle=True, num_workers=4)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
# class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create Model

In [32]:
# Create model here 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

model_ft = models.resnet50(weights='IMAGENET1K_V2')
num_classes=120
num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, num_classes)
model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),  # Additional linear layer with 256 output features
    nn.LeakyReLU(negative_slope=0.01, inplace=True),         # Activation function (you can choose other activation functions too)
    nn.Dropout(0.2),               # Dropout layer with 20% probability
    nn.Linear(256, num_classes)    # Final prediction fc layer
)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_ft.fc.parameters(), lr=0.0001, weight_decay=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
for param in model_ft.parameters():
    param.requires_grad = False
    
for param in model_ft.fc.parameters():
    param.requires_grad = True

In [33]:
#Load trained model:
model_ft = models.resnet50()
num_classes=120
num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, num_classes)
model_ft.fc = nn.Sequential(
    nn.Linear(num_ftrs, 256),  # Additional linear layer with 256 output features
    nn.LeakyReLU(negative_slope=0.01, inplace=True),         # Activation function (you can choose other activation functions too)
    nn.Dropout(0.2),               # Dropout layer with 20% probability
    nn.Linear(256, num_classes)    # Final prediction fc layer
)
model_ft.load_state_dict(torch.load('saved_model.pth'))
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_ft.fc.parameters(), lr=0.0001, weight_decay=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
for param in model_ft.parameters():
    param.requires_grad = False
    
for param in model_ft.fc.parameters():
    param.requires_grad = True


In [34]:
batch_size = 128
# Pytorch train and test sets
#train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
min_valid_loss = np.inf

# data loader
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = False)
valid_loader= DataLoader(valid_data, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [36]:
def combine_predictions(predictions):
    # Assuming predictions is a list of tensors
    # Combine predictions by taking the average
    combined = torch.stack(predictions, dim=0)
    averaged_predictions = torch.mean(combined, dim=0)
    
    # Round the averaged predictions to the nearest whole number
    rounded_predictions = torch.round(averaged_predictions)
    
    return rounded_predictions
def train_model(model, optimizer, train_loader, valid_loader, loss_module, num_epochs=100):
    min_valid_loss = np.inf
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        train_loss = 0.0
        # Set model to train mode
        model.train()
        for data_inputs, data_labels in train_loader:

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels = np.array(data_labels)
            data_labels = torch.from_numpy(data_labels)
            data_labels = data_labels.to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)
            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()
            
            ## Step 6: Add to loss
            train_loss += loss.item()
        
        #Validation Loop
        valid_loss = 0.0
        model.eval()     
        for data_inputs, data_labels in valid_loader:
            data_inputs = data_inputs.to(device)
            data_labels = np.array(data_labels)
            data_labels = torch.from_numpy(data_labels)
            data_labels = data_labels.to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)
            valid_loss += loss.item()

        print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(valid_loader)}')
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
            min_valid_loss = valid_loss
            # Saving State Dict
            torch.save(model.state_dict(), 'saved_model.pth')
train_model(model_ft, optimizer_ft, train_loader, valid_loader, criterion)
#plt.plot(loss_values)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1 		 Training Loss: 4.777501841386159 		 Validation Loss: 4.7855046689510345
Validation Loss Decreased(inf--->76.568075) 	 Saving The Model


  1%|          | 1/100 [01:37<2:40:08, 97.06s/it]

Epoch 2 		 Training Loss: 9.507806241512299 		 Validation Loss: 4.705149203538895
Validation Loss Decreased(76.568075--->75.282387) 	 Saving The Model


  2%|▏         | 2/100 [03:07<2:35:10, 95.00s/it]

...

Epoch 26 		 Training Loss: 97.32741657396157 		 Validation Loss: 2.9288795739412308
Validation Loss Decreased(47.461454--->46.862073) 	 Saving The Model


 26%|██▌       | 26/100 [39:10<1:50:57, 89.96s/it]

Epoch 27 		 Training Loss: 100.20382269720237 		 Validation Loss: 2.894901230931282
Validation Loss Decreased(46.862073--->46.318420) 	 Saving The Model


 27%|██▋       | 27/100 [41:21<1:51:48, 91.90s/it]


KeyboardInterrupt: 

In [35]:
import torch.nn.functional as F

def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs, data_labels in data_loader:

            # Determine prediction of model on dev set
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)
            preds = model(data_inputs)
            preds = F.softmax(preds, dim=1)
            max_prob_labels = torch.argmax(preds, dim=1)
            
            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (max_prob_labels == data_labels).sum().item()
            num_preds += data_labels.shape[0]

    acc = true_preds / num_preds
    print(f"Accuracy of the model: {100.0*acc:4.2f}%")

# Assuming you have already defined 'model' and 'train_loader'
eval_model(model_ft, train_loader)
eval_model(model_ft, valid_loader)
eval_model(model_ft, test_loader)

Accuracy of the model: 41.74%
Accuracy of the model: 35.27%
Accuracy of the model: 34.38%


# Submission CSV

In [50]:
import pandas as pd

def generate_submission(predictions, sample_submission_path, output_path):
    """
    Generate a Kaggle submission file based on the given predictions.

    Parameters:
    - predictions: A dictionary with image ids as keys and a list of 120 probabilities as values.
    - sample_submission_path: Path to the provided sample submission file.
    - output_path: Path to save the generated submission file.
    """
    # Load the sample submission
    sample_submission = pd.read_csv(sample_submission_path)
    
    # Replace the sample probabilities with the actual predictions
    for i in range(0, len(predictions)):
        prediction = predictions[i]
        for image_id, probs in prediction.items():
            sample_submission.loc[sample_submission['id'] == image_id, sample_submission.columns[1:]] = probs

    # Save the modified sample submission as the final submission
    sample_submission.to_csv(output_path, index=False)

# Example usage

class Dataset_Interpreter_test(Dataset):
    def __init__(self, data_path, file_names, labels=None):
        self.data_path = data_path
        self.file_names = file_names
        self.labels = labels
        self.transforms = transforms
        self.resize = Resize((224,224))
        self.normalize = Normalize(means, stds)
        self.trans = transforms.Compose([   
            transforms.ToTensor()
        ])
        
    def __len__(self):
        return (len(self.file_names))
    
    def __getitem__(self, idx):
        img_name = f'{self.file_names.iloc[idx]}'
        full_address = os.path.join(self.data_path, img_name)
        image = Image.open(full_address)
        image = self.resize(image)
        image = self.trans(image)
        image = self.normalize(image)
#         if self.transforms is not None:
#             image = self.transforms(image)
        
        return image
    
files = [f for f in os.listdir(data_dir + '/test/') if os.path.isfile(f)]
files = {'id': files}
files = pd.DataFrame(files)
test_data = Dataset_Interpreter_test(data_path=data_dir+'/test/', file_names=files['id'])    
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

def test_model(model, data_loader):
    model.eval() # Set model to eval mode
    predictions = []

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs, data_labels in data_loader:

            # Determine prediction of model on dev set
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)
            preds = model(data_inputs)
            preds = F.softmax(preds, dim=1)
            predictions.append(preds)
        
    return predictions
# Assuming you have already defined 'model' and 'train_loader'
raw_predictions = test_model(model_ft, test_loader)

predictions = []
for i in range(0, len(raw_predictions)):
    prediction = raw_predictions[i]
    image_id = files.iloc[i]['id']
    image_id = image_id.split('.jpg')[0]
    predictions.append({'image_id': image_id, 'probs': prediction})
    
# predictions = {
#     '000621fb3cbb32d8935728e48679680e': [0.01, 0.02, ...],  # Replace with actual probabilities
#     # ... add more predictions
# }
generate_submission(predictions, data_dir + '/sample_submission.csv', 'my_submission.csv')

In [37]:
predictions

[]

In [38]:
raw_predictions

[]

In [39]:
test_loader

In [40]:
test_data

In [53]:
files

,id


In [57]:
files = [f for f in os.listdir(data_dir + '/test/') if os.path.isfile(f)]
files

[]

In [59]:
data_dir + '/test/'

'mliii-assignment2/test/'